In [14]:
#距离
import pandas as pd
import math

#缩放马氏距离
def trainScaledManhattan(in_vec):
    mean = in_vec.mean()
    std = in_vec.std()
    return mean,std


def classifyScaledManhattan(mean,std, q, prob_list):

    if(len(mean) != len(q)):
        print (len(mean))
        print(len(q))
        print("Be sure that both vectors are the same dimension!")
        return
    for i in range(len(mean)):
        if (std[i] == 0):
            std[i] = 1e-10

    return sum([prob_list[i] * abs(mean[i]-q[i])/std[i] for i in range(len(q))])

#欧氏距离
def trainEuclidean(in_vec):
    mean = in_vec.mean()
    return mean

def classifyEuclidean(mean,q, prob_list):
    if(len(mean) != len(q)):
        print (len(mean))
        print(len(q))
        print("Be sure that both vectors are the same dimension!")
        return
    return sum([prob_list[i]*math.sqrt((mean[i]-q[i])**2) for i in range(len(q))])

#根据距离搜索阀值
def search_threshold(scores,threshold):
    scores_asc = sorted(scores)
    #print(scores_asc)
    threshold_index = int(len(scores)*(1-threshold))
    return scores_asc[threshold_index]


#调整的余弦相似度量

from scipy import spatial
import numpy as np


def adjusted_cos_distance_matrix(size, matrix, row_column, prob_list):
    distances = np.zeros((size,size))
    if row_column == 0:
        M_u = matrix.mean(axis=1)
        #print('M_u is:',M_u)
        m_sub = matrix.as_matrix() - M_u[:,None]
        #print('m_sub is:',m_sub)
    if row_column == 1:
        M_u = matrix.df.as_matrix().T.mean(axis=1)
        m_sub = matrix.T - M_u[:,None]
    
    prob_Matrix = []
    for i in range(len(m_sub)):
        prob_Matrix.append(prob_list)
    print('m_sub is:',m_sub)
    
    m_sub = m_sub*np.array(prob_Matrix)
    print('secend m_sub is:',m_sub)
    
    for first in range(0,size):
        for sec in range(0,size):
            distance = spatial.distance.cosine(m_sub[first],m_sub[sec])
            distances[first,sec] = distance
            if distance < 0:
                print('distance matrix',distance)
    return distances


def adjusted_cos_distance(vector1,vector2,prob_list):
    M_u1 = vector1.mean()
    M_u2 = vector2.mean()
    m_sub1 = vector1 - M_u1
    m_sub2 = vector2 - M_u2    
    m_sub1 = np.multiply(np.array(m_sub1),np.array(prob_list))
    print(m_sub1)
    m_sub2 = np.multiply(np.array(m_sub2),np.array(prob_list))
    print(m_sub2)
    distance = spatial.distance.cosine(m_sub1,m_sub2)
    if distance < 0:
        print(distance)
    return distance

def cos_distance_matrix(size, matrix, row_column,prob_list):
    distances = np.zeros((size,size))
    if row_column == 0:
        m_sub = matrix.as_matrix() 
        #print('m_sub is:',m_sub)
    if row_column == 1:
        M_u = matrix.df.as_matrix().T.mean(axis=1)
        m_sub = matrix.T
    
    prob_Matrix = []
    for i in range(len(m_sub)):
        prob_Matrix.append(prob_list)
    
    m_sub = m_sub*np.array(prob_Matrix)
    
    
    for first in range(0,size):
        for sec in range(0,size):
            distance = spatial.distance.cosine(m_sub[first],m_sub[sec])
            distances[first,sec] = distance
    return distances

def cos_distance(vector1,vector2,prob_list):
    m_sub1 = vector1
    m_sub2 = vector2
    m_sub1 = np.multiply(np.array(m_sub1),np.array(prob_list))
    m_sub2 = np.multiply(np.array(m_sub2),np.array(prob_list))    
    distance = spatial.distance.cosine(m_sub1,m_sub2)
    if distance < 0:
        print(distance)
    return distance


#中心点对应的index,0.05false alarm的distance阀值
def center (subject,vectors,prob_list):    #根据前200个样本计算中心点
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:200]
    distances = adjusted_cos_distance_matrix(200,subject_vecs,0,prob_list)
    #print(distances)
    distances_sum = pd.DataFrame(distances).sum().tolist()
    #print(distances_sum)
    center_index  = distances_sum.index(min(distances_sum))
    return subject_vecs.iloc[center_index]

def cosine_threshold (center_vec,vectors,prob_list):
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:200]
    possitive_scores = []
    for i in range(len(subject_vecs)):
        distance = adjusted_cos_distance(center_vec,subject_vecs.iloc[i],prob_list)
        possitive_scores.append(distance)
    #print(sorted(possitive_scores))
    threshold = search_threshold(possitive_scores,0.05)
    return threshold
    

def test_possitive(subject,vectors,center_vec,distance_threshold,prob_list): 
    positive_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[200:400]
    possitive_scores = []
    for i in range(len(positive_vecs)):
        distance = adjusted_cos_distance(center_vec,positive_vecs.iloc[i],prob_list)
        possitive_scores.append(distance)
    #print(possitive_scores)
    negative1 = sum(possitive_scores>distance_threshold)
    #print(negative1)
    return negative1

def test_passitive(subject,vectors,center_vec,distance_threshold,prob_list):
    negative_vecs = vectors[vectors['subject']!=subject].drop(['subject'],axis=1)    
    negative_scores = []
    for i in range(len(negative_vecs)):
        distance = adjusted_cos_distance(center_vec,negative_vecs.iloc[i],prob_list)
        negative_scores.append(distance)
    #print(sorted(negative_scores))
    negative2 = sum(negative_scores<distance_threshold)
    #print(negative2)
    return negative2

#from sklearn.preprocessing import StandardScaler
#ss = StandardScaler()

def train (subject,vectors,prob_list):
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:200]  #前200个样本训练，后200个样本测试
    mean,std = trainScaledManhattan(subject_vecs)
    
    scores = []   
    for i in range(len(subject_vecs)):
        score = classifyScaledManhattan(mean,std,subject_vecs.iloc[i],prob_list)
        #print(possitive_score)
        scores.append(score)
        
    threshold = search_threshold(scores,0.05)
    #subject_vecs = ss.fit_transform(subject_vecs)    
    
    return mean,std,threshold

def test(subject,vectors,prob_list):
    mean,std,threshold = train(subject,vectors,prob_list)    #调用训练
    #print('mean:',mean)
    #print('std:',std)
    possitive_samples = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[200:400]  #正样本
    #possitive_samples = ss.fit_transform(possitive_samples)
    negative_samples = vectors[vectors['subject']!=subject].drop(['subject'],axis=1)
    #negative_samples = ss.fit_transform(negative_samples)
    
    #print(possitive_samples)
    #print(negative_samples)
    
    possitive_scores = []
    negative_scores = []
    
    for i in range(len(possitive_samples)):
        possitive_score = classifyScaledManhattan(mean,std,possitive_samples.iloc[i],prob_list)
        #print(possitive_score)
        possitive_scores.append(possitive_score)
        
    for i in range(len(negative_samples)):
        negative_score = classifyScaledManhattan(mean,std,negative_samples.iloc[i],prob_list)
        #print(negative_score)
        negative_scores.append(negative_score)
    return possitive_scores,negative_scores,threshold

#纯余弦度量
#中心点对应的index,0.05false alarm的distance阀值
def center2 (subject,vectors,prob_list):    #根据前200个样本计算中心点
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:200]
    distances = cos_distance_matrix(200,subject_vecs,0,prob_list)
    #print(distances)
    distances_sum = pd.DataFrame(distances).sum().tolist()
    #print(distances_sum)
    center_index  = distances_sum.index(min(distances_sum))
    return subject_vecs.iloc[center_index]

def cosine_threshold2 (center_vec,vectors,prob_list):
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:200]
    possitive_scores = []
    for i in range(len(subject_vecs)):
        distance = cos_distance(center_vec,subject_vecs.iloc[i],prob_list)
        possitive_scores.append(distance)
        if distance < 0:
            print('distance threshold2:',distance)
    #print(sorted(possitive_scores))
    threshold = search_threshold(possitive_scores,0.05)
    return threshold
    

def test_possitive2(subject,vectors,center_vec,distance_threshold,prob_list): 
    positive_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[200:400]
    possitive_scores = []
    for i in range(len(positive_vecs)):
        distance = cos_distance(center_vec,positive_vecs.iloc[i],prob_list)
        possitive_scores.append(distance)
        if distance < 0:
            print(distance)
    #print(possitive_scores)
    negative1 = sum(possitive_scores>distance_threshold)
    #print(negative1)
    return negative1

def test_passitive2(subject,vectors,center_vec,distance_threshold,prob_list):
    negative_vecs = vectors[vectors['subject']!=subject].drop(['subject'],axis=1)    
    negative_scores = []
    for i in range(len(negative_vecs)):
        distance = cos_distance(center_vec,negative_vecs.iloc[i],prob_list)
        negative_scores.append(distance)
        if distance < 0:
            print(distance)
    #print(sorted(negative_scores))
    negative2 = sum(negative_scores<distance_threshold)
    #print(negative2)
    return negative2

#试试欧氏距离

def train_Euclidean (subject,vectors,prob_list):
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[0:200]  #前200个样本训练，后200个样本测试
    mean = trainEuclidean(subject_vecs)
    
    scores = []   
    for i in range(len(subject_vecs)):
        score = classifyEuclidean(mean,subject_vecs.iloc[i],prob_list)
        #print(score)
        scores.append(score)
        
    threshold = search_threshold(scores,0.05)
    #subject_vecs = ss.fit_transform(subject_vecs)    
    
    return mean,threshold

def test_Euclidean(subject,vectors,prob_list):
    mean,threshold = train_Euclidean(subject,vectors,prob_list)    #调用训练
    #print('mean:',mean)
    #print('std:',std)
    possitive_samples = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[200:400]  #正样本
    #possitive_samples = ss.fit_transform(possitive_samples)
    negative_samples = vectors[vectors['subject']!=subject].drop(['subject'],axis=1)
    #negative_samples = ss.fit_transform(negative_samples)
    
    #print(possitive_samples)
    #print(negative_samples)
    
    possitive_scores = []
    negative_scores = []
    
    for i in range(len(possitive_samples)):
        possitive_score = classifyEuclidean(mean,possitive_samples.iloc[i],prob_list)
        #print(possitive_score)
        possitive_scores.append(possitive_score)
        
    for i in range(len(negative_samples)):
        negative_score = classifyEuclidean(mean,negative_samples.iloc[i],prob_list)
        #print(negative_score)
        negative_scores.append(negative_score)
    return possitive_scores,negative_scores,threshold

Hawkes模型提出的特征，表明激励有没有有，有的话，强度是多少。考虑有没有，我用训练集中的特征出现的概率p（发生率）的导数作为惩罚，即，如果训练集中这个特征基本没有值，如果测试集中，该特征有值，那么判断很可以能不是该用户，所以这种情况，需要对该特征产生的距离进行惩罚。惩罚系数为beta/p。beta控制惩罚的力度。
下面在计算距离时，每个特征的距离要乘以这个惩罚系数。

In [16]:
#求训练集中的各特征出现的概率
# inf_pen 是特征不出现，zero_counts = 0的情况，在 zero_counts = 1的基础上再加一个惩罚inf_pen, beta是惩罚项的影响比率
def prob(subject, vectors, samples_start, samples_end, beta, inf_pen = 10):
    subject_vecs = vectors[vectors['subject']==subject].drop(['subject'],axis=1)[samples_start:samples_end]
    prob_list = []
    for columns in [column for column in subject_vecs]:
        zero_counts = len(subject_vecs[subject_vecs[columns]==0])
        if zero_counts == 0:
            prob = samples_end - samples_start + inf_pen
        else:
            prob = (samples_end - samples_start)/zero_counts
        prob_list.append(prob**beta)
    return prob_list     

In [52]:
file_name = './data/CMU_exp_shrink_output_zero_2subject.xls'
#file_name = './data/CMU_exp_shrink_output_rep50_2subject.xls'
df = pd.read_excel(file_name)
vectors = df.drop(['key','session'],axis=1)
samples_start = 0
samples_end = 200
beta = 1
inf_pen = 1

subject_list = ['s002','s032','s036','s047','s052']
for subject in subject_list:
    prob_list  = prob(subject,vectors,samples_start,samples_end,beta,inf_pen)
    #print (prob_list)
    positive_scores,negative_scores,threshold = test_Euclidean(subject,vectors,prob_list)
    negative1 = sum(i > threshold for i in positive_scores)
    negative2 = sum(i < threshold for i in negative_scores)
    
    miss_rate1 = negative1/200
    miss_rate2 = negative2/1600
    miss_rate = (negative1+negative2)/1800
    
    
    print (subject+' miss_rate1 is:',miss_rate1)
    print (subject+' miss_rate2 is:',miss_rate2)
    print (subject+' miss_rate is:',miss_rate)

s002 miss_rate1 is: 0.105
s002 miss_rate2 is: 0.52125
s002 miss_rate is: 0.475
s032 miss_rate1 is: 0.05
s032 miss_rate2 is: 0.5375
s032 miss_rate is: 0.48333333333333334
s036 miss_rate1 is: 0.175
s036 miss_rate2 is: 0.000625
s036 miss_rate is: 0.02
s047 miss_rate1 is: 0.145
s047 miss_rate2 is: 0.33125
s047 miss_rate is: 0.31055555555555553
s052 miss_rate1 is: 0.105
s052 miss_rate2 is: 0.350625
s052 miss_rate is: 0.3233333333333333


s002 miss_rate1 is: 0.1
s002 miss_rate2 is: 0.525
s002 miss_rate is: 0.4777777777777778
s032 miss_rate1 is: 0.05
s032 miss_rate2 is: 0.535625
s032 miss_rate is: 0.4816666666666667
s036 miss_rate1 is: 0.175
s036 miss_rate2 is: 0.000625
s036 miss_rate is: 0.02
s047 miss_rate1 is: 0.135
s047 miss_rate2 is: 0.339375
s047 miss_rate is: 0.31666666666666665
s052 miss_rate1 is: 0.105
s052 miss_rate2 is: 0.34125
s052 miss_rate is: 0.315

In [81]:
import pandas as pd

file_name = './data/CMU_exp_shrink_output_zero_2subject.xls'
df = pd.read_excel(file_name)
vectors = df.drop(['key','session'],axis=1)

samples_start = 0
samples_end = 200
beta = 1
inf_pen = 1

subject_list = ['s002','s032','s036','s047','s052']
#subject_list = ['s002']
for subject in subject_list:
    prob_list  = prob(subject,vectors,samples_start,samples_end,beta,inf_pen)
    center_vec = center2(subject,vectors,prob_list)    
    distance_threshold =  cosine_threshold2(center_vec,vectors,prob_list)
    #print(distance_threshold)
    negative1 = test_possitive2(subject,vectors,center_vec,distance_threshold,prob_list)
    negative2 = test_passitive2(subject,vectors,center_vec,distance_threshold,prob_list)

    miss_rate1 = negative1/200
    miss_rate2 = negative2/1600
    miss_rate = (negative1+negative2)/1800
    
    print (subject+' miss_rate1 is:',miss_rate1)
    print (subject+' miss_rate2 is:',miss_rate2)
    print (subject+' miss_rate is:',miss_rate)

c:\users\jw\anaconda3\lib\site-packages\ipykernel_launcher.py:93: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


s002 miss_rate1 is: 0.085
s002 miss_rate2 is: 0.4375
s002 miss_rate is: 0.3983333333333333
s032 miss_rate1 is: 0.12
s032 miss_rate2 is: 0.725625
s032 miss_rate is: 0.6583333333333333
s036 miss_rate1 is: 0.355
s036 miss_rate2 is: 0.00375
s036 miss_rate is: 0.042777777777777776
s047 miss_rate1 is: 0.035
s047 miss_rate2 is: 0.595
s047 miss_rate is: 0.5327777777777778
s052 miss_rate1 is: 0.035
s052 miss_rate2 is: 0.2175
s052 miss_rate is: 0.19722222222222222


In [19]:
file_name = './data/CMU_exp_shrink_output_zero_2subject.xls'
#file_name = './data/CMU_exp_shrink_output_rep50_2subject.xls'
df = pd.read_excel(file_name)
vectors = df.drop(['key','session'],axis=1)

samples_start = 0
samples_end = 200
beta = 1
inf_pen = 200


subject_list = ['s002','s032','s036','s047','s052']
#subject_list = ['s002']
for subject in subject_list:
    prob_list  = prob(subject,vectors,samples_start,samples_end,beta,inf_pen)
    center_vec = center(subject,vectors,prob_list)
    distance_threshold =  cosine_threshold(center_vec,vectors,prob_list)
    print(distance_threshold)
    negative1 = test_possitive(subject,vectors,center_vec,distance_threshold,prob_list)
    negative2 = test_passitive(subject,vectors,center_vec,distance_threshold,prob_list)

    miss_rate1 = negative1/200
    miss_rate2 = negative2/1600
    miss_rate = (negative1+negative2)/1800
    
    print (subject+' miss_rate1 is:',miss_rate1)
    print (subject+' miss_rate2 is:',miss_rate2)
    print (subject+' miss_rate is:',miss_rate)

c:\users\jw\anaconda3\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


m_sub is: [[ 1.44787448 -0.75557461  0.40801282 ... -0.75557461 -0.75557461
  -0.75557461]
 [-0.7034017  -0.7034017   0.69695982 ... -0.7034017  -0.7034017
  -0.7034017 ]
 [ 2.2215668  -0.61994819  0.86150865 ... -0.61994819 -0.61994819
  -0.61994819]
 ...
 [ 3.49411711 -0.89983869  1.37198493 ... -0.89983869 -0.89983869
  -0.89983869]
 [-0.81821751 -0.81821751  1.71338113 ... -0.81821751 -0.81821751
  -0.81821751]
 [-0.74537798 -0.74537798  1.7883577  ... -0.74537798 -0.74537798
  -0.74537798]]
secend m_sub is: [[  2.54013066  -0.76708083  27.20085442 ...  -0.75937147  -0.75557461
   -0.75557461]
 [ -1.23403806  -0.7141134   46.46398828 ...  -0.70693638  -0.7034017
   -0.7034017 ]
 [  3.89748561  -0.62938903  57.43391003 ...  -0.62306351  -0.61994819
   -0.61994819]
 ...
 [  6.13003001  -0.91354182  91.4656622  ...  -0.90436049  -0.89983869
   -0.89983869]
 [ -1.43546932  -0.83067768 114.22540891 ...  -0.82232916  -0.81821751
   -0.81821751]
 [ -1.30768066  -0.75672891 119.22384685 ..

[ 5.34039743e+00 -8.91446914e-01  7.93991114e+01 -9.19450482e-01
  1.46809827e+01 -8.95995112e-01  1.44906036e+01 -8.78075210e-01
  1.50887109e+02 -8.78075210e-01  2.92037792e+02 -8.78075210e-01
  4.03880802e+02 -1.38279561e+00  3.84586539e+00 -8.86944657e-01
  3.16332397e+02 -8.78075210e-01 -8.78075210e-01  1.27017890e+03
 -1.58211750e+00 -7.21368501e+01 -2.31072424e+00 -8.82487648e-01
 -8.78075210e-01 -8.78075210e-01]
[  5.21191882  -0.87330473  78.97424377  -0.90073838  14.79540491
  -0.87776036  13.44334044  -0.86020515 200.57846485  -0.86020515
 311.99368284  -0.86020515 355.87238988  -1.35465379   2.65746586
  -0.8688941  265.62717179  -0.86020515  -0.86020515 838.04883714
   1.25303858 -82.56972677  -2.26369777  -0.86452779  -0.86020515
  -0.86020515]
[-1.28164443e+00 -7.41662261e-01  8.92704669e+01 -7.64960552e-01
  1.86270213e+01 -7.45446252e-01  2.14848313e+01 -7.30537327e-01
  2.56478720e+02 -7.30537327e-01  3.61580715e+02 -7.30537327e-01
  4.35582845e+02 -1.15045248e+00  3.

  -0.86020515]
[ 7.15302857e+00 -1.05171310e+00  1.12855679e+02 -1.08475121e+00
  2.29111684e+01 -1.05707898e+00  2.92830644e+01 -1.03593741e+00
  2.64087331e+02 -1.03593741e+00  4.09501938e+02 -1.03593741e+00
  4.87333150e+02  1.90172742e+00 -2.91813354e+00 -1.04640142e+00
  3.04599856e+02 -1.03593741e+00 -1.03593741e+00  1.09668362e+03
 -1.81165739e+00 -5.52749014e+01 -2.72615107e+00 -1.04114312e+00
 -1.03593741e+00 -1.03593741e+00]
[  5.21191882  -0.87330473  78.97424377  -0.90073838  14.79540491
  -0.87776036  13.44334044  -0.86020515 200.57846485  -0.86020515
 311.99368284  -0.86020515 355.87238988  -1.35465379   2.65746586
  -0.8688941  265.62717179  -0.86020515  -0.86020515 838.04883714
   1.25303858 -82.56972677  -2.26369777  -0.86452779  -0.86020515
  -0.86020515]
[   6.73796901   -1.21118678   95.39081214   -1.24923453   17.70007657
   -1.2173663    16.95897555   -1.19301898  222.60559147   -1.19301898
  347.93622866   -1.19301898  378.58751658    1.47756415   -3.29591414
   

 -6.58651134e-01 -6.58651134e-01]
[  5.21191882  -0.87330473  78.97424377  -0.90073838  14.79540491
  -0.87776036  13.44334044  -0.86020515 200.57846485  -0.86020515
 311.99368284  -0.86020515 355.87238988  -1.35465379   2.65746586
  -0.8688941  265.62717179  -0.86020515  -0.86020515 838.04883714
   1.25303858 -82.56972677  -2.26369777  -0.86452779  -0.86020515
  -0.86020515]
[-1.26567806e+00 -7.32422834e-01  9.16225651e+01  1.47538216e+00
 -1.80359123e+01 -7.36159685e-01  2.02810091e+01 -7.21436491e-01
  1.50877766e+02 -7.21436491e-01  1.51452121e+02 -7.21436491e-01
  1.20981926e+02 -1.13612046e+00  9.76760772e-01 -7.28723729e-01
  8.42726882e+01 -7.21436491e-01 -7.21436491e-01 -1.63618012e+01
  9.32639942e+00 -8.68272805e+00 -7.57135359e-02 -7.25061800e-01
 -7.21436491e-01 -7.21436491e-01]
[  5.21191882  -0.87330473  78.97424377  -0.90073838  14.79540491
  -0.87776036  13.44334044  -0.86020515 200.57846485  -0.86020515
 311.99368284  -0.86020515 355.87238988  -1.35465379   2.65746586

...

 -8.20144493e-01 -7.91439436e-01]
[-1.21021404e+00 -9.24384596e-01  5.81383381e+01  2.54930970e+00
 -2.42042809e+00 -9.58086118e-01  1.43231025e+01 -9.19762673e-01
  3.72963479e+02 -9.19762673e-01  4.74819926e+02 -1.06949148e+00
  1.04415777e+01 -9.88992122e-01  2.15884760e+01 -9.19762673e-01
  7.19785013e+02 -9.33769211e-01 -9.19762673e-01  1.27209821e+03
 -1.25994887e+00 -3.90624081e+01 -4.47465375e-01 -9.43346331e-01
 -9.53121941e-01 -9.19762673e-01]
[-9.27624068e-01 -7.08536977e-01  5.26461033e+01  2.49175362e+00
 -1.85524814e+00 -7.34369054e-01  1.37999793e+01 -7.04994292e-01
  1.82383272e+02 -7.04994292e-01  2.18122396e+02 -8.19760805e-01
  4.24535803e+00 -7.58058379e-01  8.63632821e+00 -7.04994292e-01
  6.01801718e+02 -7.15730246e-01 -7.04994292e-01  1.20855507e+03
 -9.65745606e-01  3.03999489e+01  2.84762567e-01 -7.23071069e-01
 -7.30564033e-01 -7.04994292e-01]
[-1.21021404e+00 -9.24384596e-01  5.81383381e+01  2.54930970e+00
 -2.42042809e+00 -9.58086118e-01  1.43231025e+01 -9.1

 -9.53121941e-01 -9.19762673e-01]
[-7.95311476e-01 -6.07474092e-01  4.98231844e+01 -9.37111197e-01
  2.30780538e+00 -6.29621585e-01  1.54548697e+01 -6.04436722e-01
  1.86514053e+02 -6.04436722e-01  2.42283847e+02 -7.02833398e-01
  4.75791031e+00 -6.49931959e-01  1.12134588e+01 -6.04436722e-01
  3.29761650e+02 -6.13641342e-01 -6.04436722e-01  1.03790345e+03
 -8.27995510e-01  3.09082161e+01  4.15417549e-01 -6.19935099e-01
 -6.26359297e-01 -6.04436722e-01]
[-1.21021404e+00 -9.24384596e-01  5.81383381e+01  2.54930970e+00
 -2.42042809e+00 -9.58086118e-01  1.43231025e+01 -9.19762673e-01
  3.72963479e+02 -9.19762673e-01  4.74819926e+02 -1.06949148e+00
  1.04415777e+01 -9.88992122e-01  2.15884760e+01 -9.19762673e-01
  7.19785013e+02 -9.33769211e-01 -9.19762673e-01  1.27209821e+03
 -1.25994887e+00 -3.90624081e+01 -4.47465375e-01 -9.43346331e-01
 -9.53121941e-01 -9.19762673e-01]
[-8.98403377e-01 -6.86217655e-01  4.69128883e+01 -1.05858382e+00
  2.38230729e+00 -7.11236007e-01  1.44213490e+01 -6.8

[-1.21021404e+00 -9.24384596e-01  5.81383381e+01  2.54930970e+00
 -2.42042809e+00 -9.58086118e-01  1.43231025e+01 -9.19762673e-01
  3.72963479e+02 -9.19762673e-01  4.74819926e+02 -1.06949148e+00
  1.04415777e+01 -9.88992122e-01  2.15884760e+01 -9.19762673e-01
  7.19785013e+02 -9.33769211e-01 -9.19762673e-01  1.27209821e+03
 -1.25994887e+00 -3.90624081e+01 -4.47465375e-01 -9.43346331e-01
 -9.53121941e-01 -9.19762673e-01]
[ 2.75188637e+00 -7.91979467e-01  2.35215582e+01 -1.22173577e+00
  7.01892115e-01 -8.20853718e-01  4.20644982e-01 -7.88019570e-01
  1.69538696e+01 -7.88019570e-01  9.13221754e+01 -9.16301825e-01
  5.29332758e+00 -8.47332871e-01  1.37443014e+01 -7.88019570e-01
  4.79379208e+02 -8.00019868e-01 -7.88019570e-01  6.31099346e+02
  4.85166386e+00 -1.25461059e+02 -4.04112600e+00 -8.08225200e-01
 -8.16600590e-01 -7.88019570e-01]
[-1.21021404e+00 -9.24384596e-01  5.81383381e+01  2.54930970e+00
 -2.42042809e+00 -9.58086118e-01  1.43231025e+01 -9.19762673e-01
  3.72963479e+02 -9.19

 -9.53121941e-01 -9.19762673e-01]
[-1.00962516e+00 -7.71170976e-01  5.22819028e+01 -1.18963585e+00
  2.32096549e+00 -7.99286584e-01  1.30700527e+01 -7.67315121e-01
  2.75425779e+02 -7.67315121e-01  3.61831519e+02 -8.92226885e-01
  8.15132868e+00 -8.25070022e-01  1.93591380e+01 -7.67315121e-01
  6.02355329e+02 -7.79000123e-01 -7.67315121e-01  1.19853693e+03
 -1.05111660e+00 -5.46066840e+00 -4.05395430e-03 -7.86989868e-01
 -7.95145203e-01 -7.67315121e-01]
[-1.21021404e+00 -9.24384596e-01  5.81383381e+01  2.54930970e+00
 -2.42042809e+00 -9.58086118e-01  1.43231025e+01 -9.19762673e-01
  3.72963479e+02 -9.19762673e-01  4.74819926e+02 -1.06949148e+00
  1.04415777e+01 -9.88992122e-01  2.15884760e+01 -9.19762673e-01
  7.19785013e+02 -9.33769211e-01 -9.19762673e-01  1.27209821e+03
 -1.25994887e+00 -3.90624081e+01 -4.47465375e-01 -9.43346331e-01
 -9.53121941e-01 -9.19762673e-01]
[ 4.23835711e+00 -8.58181948e-01  4.06492826e+01 -1.32386208e+00
  1.68255095e+00 -8.89469832e-01  8.23233665e+00 -8.5

KeyboardInterrupt: 

In [13]:
a = spatial.distance.cosine([1,10000000,1,100000],[1,1,1,1])
a

-121.96928946530988